In [26]:
import pandas as pd             
import numpy as np

import seaborn as sns
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})   

import matplotlib.pyplot as plt
%matplotlib inline

import warnings 
warnings.filterwarnings("ignore")

import sklearn
from sklearn import linear_model
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score

from aux_fun import evaluate
from aux_fun import create_submission_dataframe

import csv

In [27]:
df = pd.read_csv('./data/train_imputed.csv')

In [28]:
df.shape

(523021, 45)

In [29]:
df.head()

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,NearestCompetitor,Region,NumberOfCustomers,NumberOfSales,Region_AreaKM2,...,Max_VisibilityKm,Min_VisibilitykM,Mean_VisibilityKm,Hyper_Market,Shopping_Center,Standard_Market,Super_Market,General,With_Fish_Department,With_Non-Food_Department
0,1000,2016-01-03,0,1,0,326,7,495,5676,9643,...,19.0,6.0,11.0,1,0,0,0,1,0,0
1,1000,2016-02-03,0,1,0,326,7,608,8111,9643,...,23.0,10.0,13.0,1,0,0,0,1,0,0
2,1000,2016-04-03,0,1,0,326,7,665,8300,9643,...,31.0,8.0,11.0,1,0,0,0,1,0,0
3,1000,2016-05-03,0,1,0,326,7,630,7154,9643,...,31.0,10.0,15.0,1,0,0,0,1,0,0
4,1000,2016-06-03,0,0,0,326,7,0,0,9643,...,31.0,5.0,12.0,1,0,0,0,1,0,0


In [30]:
df.columns

Index(['StoreID', 'Date', 'IsHoliday', 'IsOpen', 'HasPromotions',
       'NearestCompetitor', 'Region', 'NumberOfCustomers', 'NumberOfSales',
       'Region_AreaKM2', 'Region_GDP', 'Region_PopulationK', 'Max_Dew_PointC',
       'Max_Gust_SpeedKm_h', 'Max_Humidity', 'Max_Sea_Level_PressurehPa',
       'Max_TemperatureC', 'Max_Wind_SpeedKm_h', 'Mean_Dew_PointC',
       'Mean_Humidity', 'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC',
       'Mean_Wind_SpeedKm_h', 'Min_Dew_PointC', 'Min_Humidity',
       'Min_Sea_Level_PressurehPa', 'Min_TemperatureC', 'Precipitationmm',
       'WindDirDegrees', 'CloudCover', 'Fog', 'Hail', 'Rain', 'Snow',
       'Thunderstorm', 'Max_VisibilityKm', 'Min_VisibilitykM',
       'Mean_VisibilityKm', 'Hyper_Market', 'Shopping_Center',
       'Standard_Market', 'Super_Market', 'General', 'With_Fish_Department',
       'With_Non-Food_Department'],
      dtype='object')

In [31]:
y = df['NumberOfSales']

In [32]:
X = df.drop(df[['NumberOfSales', 'WindDirDegrees']], axis=1)

# Train-Test Split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
dates_test = X_test['Date']

In [35]:
X_train = X_train.drop(['Date'], axis=1)
X_test = X_test.drop(['Date'], axis=1)

# Feature Selection

## NOT WORKING

In [14]:
rfe = RFE(linear_model.LinearRegression(), 3) #10 is the number of features to consider
fit = rfe.fit(X_train, y_train)

In [60]:
feature_mask = fit.get_support() #mask of the chosen features
feature_mask

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True,  True,  True,  True, False], dtype=bool)

In [61]:
feature_selected =X_train.columns[feature_mask]
feature_selected

Index(['Hyper_Market', 'Shopping_Center', 'Standard_Market', 'Super_Market',
       'General', 'With_Fish_Department'],
      dtype='object')

# Model Creation

## OLS

In [36]:
X_train.columns

Index(['StoreID', 'IsHoliday', 'IsOpen', 'HasPromotions', 'NearestCompetitor',
       'Region', 'NumberOfCustomers', 'Region_AreaKM2', 'Region_GDP',
       'Region_PopulationK', 'Max_Dew_PointC', 'Max_Gust_SpeedKm_h',
       'Max_Humidity', 'Max_Sea_Level_PressurehPa', 'Max_TemperatureC',
       'Max_Wind_SpeedKm_h', 'Mean_Dew_PointC', 'Mean_Humidity',
       'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC',
       'Mean_Wind_SpeedKm_h', 'Min_Dew_PointC', 'Min_Humidity',
       'Min_Sea_Level_PressurehPa', 'Min_TemperatureC', 'Precipitationmm',
       'CloudCover', 'Fog', 'Hail', 'Rain', 'Snow', 'Thunderstorm',
       'Max_VisibilityKm', 'Min_VisibilitykM', 'Mean_VisibilityKm',
       'Hyper_Market', 'Shopping_Center', 'Standard_Market', 'Super_Market',
       'General', 'With_Fish_Department', 'With_Non-Food_Department'],
      dtype='object')

In [37]:
regr = linear_model.LinearRegression()

In [38]:
regr.fit(X_train['NumberOfCustomers'].reshape(-1, 1), y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [39]:
y_pred = regr.predict(X_test['NumberOfCustomers'].reshape(-1, 1))

In [40]:
print('Coefficients: \n', regr.coef_)
#The mean squared error

Coefficients: 
 [ 13.3448609]


## Lasso

In [41]:
lasso = linear_model.Lasso(0.01, normalize=True)

In [42]:
lasso.fit(X_train, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [43]:
y_pred_lasso = lasso.predict(X_test)

In [44]:
y_pred_lasso

array([ 4188.84530665,   -66.9666144 ,  4806.47294474, ...,  6102.82701915,
        6395.80065768,  -409.50500737])

## Lasso CV

In [46]:
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1]

In [47]:
lassocv = linear_model.LassoCV(eps=0.001, n_alphas=100, alphas=alphas, fit_intercept=True, normalize=False, max_iter=10000,  cv=10)

In [48]:
lassocv.fit(X_train, y_train)

LassoCV(alphas=[1e-05, 0.0001, 0.001, 0.01, 0.1], copy_X=True, cv=10,
    eps=0.001, fit_intercept=True, max_iter=10000, n_alphas=100, n_jobs=1,
    normalize=False, positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

In [49]:
y_pred_lassocv = lassocv.predict(X_test)

In [50]:
lassocv.alpha_

0.01

# Creation of the submission 

In [51]:
result = pd.DataFrame(X_test['StoreID'])
result['Month'] = pd.DatetimeIndex(dates_test).month
result['NumberOfSales'] = y_pred
#Group by Month
result =result.groupby(['StoreID','Month'], as_index=False)['NumberOfSales'].sum()

In [66]:
result_lasso = pd.DataFrame(X_test['StoreID'])
result_lasso['Month'] = pd.DatetimeIndex(dates_test).month
result_lasso['NumberOfSales'] = y_pred_lasso
#Group by Month
result_lasso =result_lasso.groupby(['StoreID','Month'], as_index=False)['NumberOfSales'].sum()

In [67]:
result_lassocv = pd.DataFrame(X_test['StoreID'])
result_lassocv['Month'] = pd.DatetimeIndex(dates_test).month
result_lassocv['NumberOfSales'] = y_pred_lassocv
#Group by Month
result_lassocv =result_lassocv.groupby(['StoreID','Month'], as_index=False)['NumberOfSales'].sum()

In [68]:
X_test['NumberOfSales']= y_test
X_test['Date'] = dates_test

In [69]:
X_test.head()

,StoreID,IsHoliday,IsOpen,HasPromotions,NearestCompetitor,Region,NumberOfCustomers,Region_AreaKM2,Region_GDP,Region_PopulationK,...,Hyper_Market,Shopping_Center,Standard_Market,Super_Market,General,With_Fish_Department,With_Non-Food_Department,NumberOfSales,Date,Month
199480,1284,0,1,1,1145,6,245,16327,15931,4911,...,1,0,0,0,1,0,0,3452,2017-02-21,2
492279,1705,0,0,0,3482,2,0,32221,16186,5727,...,1,0,0,0,1,0,0,0,2016-04-17,4
500907,1717,0,1,1,3029,2,252,32221,16186,5727,...,1,0,0,0,0,0,1,4679,2017-06-23,6
63763,1091,0,1,0,7337,9,351,15566,15017,8146,...,1,0,0,0,0,0,1,5232,2016-01-03,1
74453,1107,0,0,0,22406,1,0,7385,9893,1018,...,0,0,1,0,0,0,1,0,2016-10-07,10


In [70]:
y_pred - y_pred_lasso

array([ -321.68725815,   664.63374178,  -845.90086991, ...,  -247.28469616,
       -1247.53596253,  1007.17213475])

### Error function given

In [71]:
eval_lr =evaluate(X_test,result)
eval_lasso = evaluate(X_test,result_lasso)
eval_lassocv = evaluate(X_test,result_lassocv)

In [73]:
print('Linear Regression Error on Number of Customers: ', eval_lr)
print('Lasso Error: ',  eval_lasso)
print('Lasso CV Error: ', eval_lassocv)

Linear Regression Error on Number of Customers:  0.0941253447948
Lasso Error:  0.0717925372867
Lasso CV Error:  0.0718105819342


### R2

In [76]:
r2_lr = r2_score(y_test, y_pred)
r2_lasso =r2_score(y_test, y_pred_lasso)
r2_lassocv = r2_score(y_test, y_pred_lassocv)

In [77]:
print('Linear Regression R2: ', r2_lr)
print('Lasso R2: ',  r2_lasso)
print('Lasso CV R2: ', r2_lassocv)

Linear Regression R2:  0.828704609738
Lasso R2:  0.909973466579
Lasso CV R2:  0.91020900379
